# Preparação da tabela de OP

### Importando as bibliotecas necessárias

In [ ]:
import numpy as np
import pandas as pd
from resolve_path import ajuste_path, read_input

### Lendo o dataframe de OP

In [ ]:
path = "data/"
pathIn = path + "input/"

pathUtil = path + "util/"
pathUtil = ajuste_path(pathUtil)

df_original = pd.read_csv(
    pathUtil + "os/IW47_Executadas_processado.csv", sep="|", on_bad_lines='skip')

df_coord = pd.read_csv(pathUtil + "local_coordenada.csv", sep=";")

df_copy_ori = df_original.copy()
df_copy_coord = df_coord.copy()

In [ ]:
df_original = df_copy_ori
df_coord = df_copy_coord

print(df_original.info())
print(df_coord.info())

### Adiciona coluna TAM (Tipo de atividade)

Leitura das tabelas ZPM e Suporte-TAM

In [ ]:
df_tam = read_input("Suporte-TAM.csv")

df_zpm = read_input("ZPM_DATAS_ORDEM_Executadas-2020-até-28jun24.csv")

Observação: 
Todas as ordens presentes na tabela IW47 estão contidas na tabela ZPM.

Mas atualmente, (06/08/24), existem 9196 ordens de 80809 ordens totais na tabela ZPM que não estão contidas na IW47. 

In [ ]:
# Converte as colunas de interesse para string
df_original["Ordem"] = df_original["Ordem"].astype(str)
df_zpm["Nº ordem"] = df_zpm["Nº ordem"].astype(str)
df_tam["TAM"] = df_tam["TAM"].astype(str)
df_zpm["Tipo de atividade"] = df_zpm["Tipo de atividade"].astype(str)

# Adicionando a coluna 'Denominação TAM' do df_tam ao df_zpm
df_zpm_tam = pd.merge(
    df_zpm, df_tam, left_on='Tipo de atividade', right_on='TAM', how='left')

# Adiciona as colunas 'TAM' e 'Denominação TAM' do df_zpm_tam ao df_original
df_original = pd.merge(df_original, df_zpm_tam[[
                       "Nº ordem", "TAM", "Denominação TAM"]], left_on='Ordem', right_on='Nº ordem', how='left')

### Removendo colunas que não serão usadas

In [ ]:
df_original = df_original[
    [
        'Ordem',
        'Operação',
        'Suboperação',
        'Confirmação',
        'Nº pessoal',
        'Tipo ordem',
        'TAM',
        'Denominação TAM',
        'Txt.breve operação',
        'Local de instalação',
        'Trabalho real',
        'Unid.trabalho (real)',
        'Trabalho (planejado)',
        'Unidade trab.(plan.)',
        'Data do início real',
        'Data do fim real',
        'Data fim prevista',
        'estornado',
        'Confirmação estorn.'
    ]
].reset_index()

df_original.info()

### Tirando OP com HH igual a 0 e removendo confirmações extornadas

In [ ]:
# df_original = df_original.loc[
#     df_original["Trabalho real"] != 0
# ]

df_original = df_original[df_original['estornado'] != 'X']
df_original = df_original[df_original['Confirmação estorn.'] == 0]

colunas_estorno = [
    'estornado',
    'Confirmação estorn.'
]

df_original.drop(columns=colunas_estorno, axis=1, inplace=True)

### Remove linhas com 'txt.breve operação' igual a 'relatorio' e 'planejamento' pois não oferecem risco

In [ ]:
# Remove linhas com 'txt.breve operação' igual a 'relatorio' e 'planejamento'
df_original = df_original[~df_original["Txt.breve operação"].isin(
    ["relatorio", "planejamento"])]

### Removendo OP sem Local de Instalação

In [ ]:
df_original = df_original.loc[df_original["Local de instalação"] != ""]
df_original['Local de instalação'] = df_original['Local de instalação'].dropna()

### Reduzindo a quantidade de siglas do Local de Instalação

In [ ]:
def local(linha):
    string = str(linha)
    parts = string.split("-")

    if len(parts) > 3:
        result = "-".join(parts[:3])
    else:
        result = string

    return result


df_original["Local de instalação"] = df_original["Local de instalação"].apply(
    local)
df_coord["Local de instalação"] = df_coord["Local de instalação"].apply(local)

### Removendo OP com Local de instalação sem coordenada

In [ ]:
print(df_original["Local de instalação"].nunique())
print(df_coord["Local de instalação"].nunique())

locais_nao_presentes = df_coord[
    ~df_coord["Local de instalação"].isin(df_original["Local de instalação"])
]
print("Locais nao presentes: ")
print(locais_nao_presentes["Local de instalação"].unique())

operacoes_nao_presentes = df_original[~df_original["Local de instalação"].isin(
    df_coord["Local de instalação"])]

print("operacoes nao presentes:")
print(operacoes_nao_presentes["Local de instalação"].unique())

df_original = df_original[
    df_original["Local de instalação"].isin(df_coord["Local de instalação"])
]

print(df_original["Local de instalação"].nunique())

### Reduzindo os Txt.breve Operacao e agrupando os com menos frequência em Outros

In [ ]:
def top_n(df, n, coluna):
    top = df[coluna].value_counts().head(n).index

    df.loc[~df[coluna].isin(top), coluna] = "outros"

    return df


df_original = top_n(df_original, 5, "Txt.breve operação")

### Agrupando TAMs com o mesmo significado

In [ ]:
df_original.loc[df_original['Denominação TAM'] ==
                'Melhorias', 'Denominação TAM'] = 'MELHORIA'
df_original.loc[df_original['TAM'] == 470, 'TAM'] = 260

df_original.loc[df_original['Denominação TAM'] ==
                'REPARO DE FALHA', 'Denominação TAM'] = 'REPARO DEFEITO'
df_original.loc[df_original['TAM'] == 300, 'TAM'] = 290

df_original.loc[df_original['Denominação TAM'] ==
                'Ensaios gerais', 'Denominação TAM'] = 'ENSAIOS'
df_original.loc[df_original['TAM'] == 410, 'TAM'] = 150

### Reduzindo os TAMs e agrupando os com menos frequência em Outros

In [ ]:
df_original = top_n(df_original, 10, "TAM")

### Criando Data de início e fim

In [ ]:
df_original["Data inicio"] = df_original["Data do início real"]

df_original["Data fim"] = df_original["Data do fim real"]

colunas_data = [
    "Data do início real",
    "Data do fim real",
    "Data fim prevista"
]

df_original.drop(columns=colunas_data, axis=1, inplace=True)

### Criando a coluna do HH final do agrupamento

In [ ]:
def hh_final(linha):
    if linha["Trabalho real"] == 0:
        return linha["Trabalho (planejado)"]
    else:
        return linha["Trabalho real"]


df_original["HH final"] = df_original.apply(hh_final, axis=1)

colunas_trabalho = [
    "Trabalho (planejado)",
    "Trabalho real"
]

df_original.drop(columns=colunas_trabalho, axis=1, inplace=True)

### Trocando os HH em minutos para horas

In [ ]:
df_original.loc[df_original['Unid.trabalho (real)'] == "MIN", "HH final"] /= 60
df_original.loc[df_original['Unid.trabalho (real)']
                == "MIN", "Unid.trabalho (real)"] = "HR"

### Montando a tabela com as colunas necessárias

In [ ]:
df_cortado = df_original[
    [
        "Ordem",
        "Operação",
        'Suboperação',
        "Txt.breve operação",
        "Local de instalação",
        "Data inicio",
        "Data fim",
        "HH final",
        "TAM",
        "Denominação TAM",
        "Confirmação",
    ]
]
df = df_cortado.sort_values(by=["Data inicio", "Local de instalação"])

print(df.head(5))

### Exportando o DataFrame

In [ ]:
df.to_csv(pathUtil + "os/IW47_Executadas_preparado.csv", index=False)